In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install --upgrade xlrd #colab bug verson xlrd
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install pygeos
!pip install topojson
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a

In [2]:
#test_1 = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/5f71ba43-afc8-43a0-b306-dafe29940f9c", sep = ";", encoding = 'utf-8', engine = 'python')
#le lien du dossier data.gouv : 
# https://www.data.gouv.fr/fr/datasets/effectif-de-patients-par-pathologie-sexe-classe-dage-et-territoire-departement-region/
# methode "classique" pour obtenir un dataset a partir d un lien 
# probleme d encodage d ou les parametres sep, encoding, engine
# A FAIRE : COPIER LA BASE DE FACON LOCALE ET S Y REFERER PAR LA SUITE PAR EFFECTIFS.CSV 

In [3]:
test_2 = pd.read_csv("effectifs.csv", sep = ";", encoding = 'utf-8', engine = 'python')
#commande a utiliser une fois qu on a la base 
#copiee en local 
#la base au prealable telechargee sur ordi perso

In [4]:
test_2.head()

,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,dept,ntop,npop,prev,niveau_prioritaire,libelle_classe_age,libelle_sexe,tri
0,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,75,999,230570.0,393250.0,58.632,1,de 45 à 49 ans,tous sexes,17.0
1,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,11,14220.0,24330.0,58.451,1,de 45 à 49 ans,tous sexes,17.0
2,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,12,10630.0,17390.0,61.133,1,de 45 à 49 ans,tous sexes,17.0
3,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,30,30760.0,50820.0,60.539,1,de 45 à 49 ans,tous sexes,17.0
4,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,48,3000.0,4920.0,60.898,1,de 45 à 49 ans,tous sexes,17.0


In [5]:
len(test_2['patho_niv1'].unique())

18

In [6]:
print("nombre de categories differentes pour niv2 est :"+str(len(test_2['patho_niv2'].unique())))
print("nombre de categories differentes pour niv3 est :"+str(len(test_2['patho_niv3'].unique())))

nombre de categories differentes pour niv2 est :46
nombre de categories differentes pour niv3 est :59


In [ ]:
#potentiellement creer 123 colonnes pour pouvoir faire du clustering 
#comparer les clusters obtenus par les caracteristiques d un inidivu pour voir si ca matche ou pas 


In [7]:
data_gen = test_2

In [8]:
data_gen.head()

,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,dept,ntop,npop,prev,niveau_prioritaire,libelle_classe_age,libelle_sexe,tri
0,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,75,999,230570.0,393250.0,58.632,1,de 45 à 49 ans,tous sexes,17.0
1,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,11,14220.0,24330.0,58.451,1,de 45 à 49 ans,tous sexes,17.0
2,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,12,10630.0,17390.0,61.133,1,de 45 à 49 ans,tous sexes,17.0
3,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,30,30760.0,50820.0,60.539,1,de 45 à 49 ans,tous sexes,17.0
4,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,48,3000.0,4920.0,60.898,1,de 45 à 49 ans,tous sexes,17.0


In [10]:
liste_diff_patho1 = data_gen['patho_niv1'].unique()
liste_diff_patho2 = data_gen['patho_niv2'].unique()
liste_diff_patho3 = data_gen['patho_niv3'].unique()

print(liste_diff_patho1, "\n", liste_diff_patho2, "\n", liste_diff_patho3)

['Pas de pathologies repérées, traitements, maternité, hospitalisations ni traitement antalgique ou anti-inflammatoire'
 'Séjours en hospitalisation complète pour prise en charge de la Covid-19'
 'Traitements du risque vasculaire (hors pathologies)'
 'Maladies inflammatoires ou rares ou VIH ou SIDA'
 'Maladies neurologiques ou dégénératives'
 'Traitements psychotropes (hors pathologies)'
 'Traitement antalgique ou anti-inflammatoire (hors pathologies, traitements, maternité ou hospitalisations)'
 'Maladies psychiatriques' nan
 'Autres affections de longue durée (dont 31 et 32)' 'Cancers'
 'Maladies cardio-neurovasculaires'
 'Maladies du foie ou du pancréas (hors mucoviscidose)' 'Diabète'
 'Hospitalisations ponctuelles (avec ou sans pathologies, traitements ou maternité)'
 'Maladies respiratoires chroniques (hors mucoviscidose)'
 'Maternité (avec ou sans pathologies)'
 'Insuffisance rénale chronique terminale'] 
 [nan
 'Séjours en hospitalisation complète pour prise en charge de la Covi

In [15]:
data_gen.shape

(4057201, 16)

In [43]:
#Creation d une colonne individus
ind = []
for i in range (1,4057202) : 
    ind.append(i)

In [44]:
ind = pd.DataFrame(ind)
ind = ind.set_axis(['ind'], axis='columns')

In [45]:
data_gen_1 = pd.concat([ind, data_gen], axis = 1)

In [46]:
data_gen_1.head()

,ind,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,dept,ntop,npop,prev,niveau_prioritaire,libelle_classe_age,libelle_sexe,tri
0,1,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,75,999,230570.0,393250.0,58.632,1,de 45 à 49 ans,tous sexes,17.0
1,2,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,11,14220.0,24330.0,58.451,1,de 45 à 49 ans,tous sexes,17.0
2,3,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,12,10630.0,17390.0,61.133,1,de 45 à 49 ans,tous sexes,17.0
3,4,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,30,30760.0,50820.0,60.539,1,de 45 à 49 ans,tous sexes,17.0
4,5,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,48,3000.0,4920.0,60.898,1,de 45 à 49 ans,tous sexes,17.0


In [66]:
#tentative de compréhension de ce que la variable npop représente
b = data_gen['npop'].unique()
b = pd.DataFrame(b)

In [59]:
b = b.dropna()

In [63]:
b = b.set_axis(['test'], axis = 'columns')

In [64]:
b.head()

,test
0,393250.0
1,24330.0
2,17390.0
3,50820.0
4,4920.0


In [65]:
sum(b['test'])

4734851240.0

In [67]:
#resultat peu concluant